# Damage detection and recognition training pipeline

### Define run parameters
The dataset should follow the VGGFace2/ImageNet-style directory layout. Modify data_dir to the location of the dataset on wish to finetune on.

In [1]:
from facenet_pytorch import InceptionResnetV1, fixed_image_standardization, training
import torch
from torch.utils.data import Dataset, DataLoader, SubsetRandomSampler
from torch import optim
from torch.optim.lr_scheduler import MultiStepLR
from torch.utils.tensorboard import SummaryWriter
from torchvision import datasets, transforms
import numpy as np
import os
# from PIL import Image
# import copy
# import random
from functions import SiameseInceptionResnetV1, siamese_dataset, pass_epoch

In [2]:
data_dir = "C:/Users/U339700/Documents/Palas/dataset/prueba_siamesas/crop_2class_train"
val_dir = "C:/Users/U339700/Documents/Palas/dataset/prueba_siamesas/crop_2class_val"
model_dir = 'C:/Users/U339700/Documents/Palas/models'

batch_size = 32
epochs = 8
save_after = 1
workers = 0 if os.name == 'nt' else 8

### Determine if an nvidia GPU is available

In [3]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Running on device: {}'.format(device))

Running on device: cpu


### Define Inception Resnet V1 module
See help(InceptionResnetV1) for more details.

Define dataset and dataloader

In [4]:
trans = transforms.Compose([
    transforms.Resize((256, 256)),
    np.float32,
    transforms.ToTensor(),
    fixed_image_standardization
])

data_augm_horz = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.RandomHorizontalFlip(p=1),
    np.float32,
    transforms.ToTensor(),
    fixed_image_standardization
])

data_augm_vert = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.RandomVerticalFlip(p=1),
    transforms.ToTensor(),
    fixed_image_standardization
])

data_augm_persp = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.RandomPerspective(distortion_scale=0.6, p=1.0),
    np.float32,
    transforms.ToTensor(),
    fixed_image_standardization
])

data_augm_eq = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.RandomEqualize(p=1),
    np.float32,
    transforms.ToTensor(),
    fixed_image_standardization
])

data_augm_blur = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.GaussianBlur(kernel_size=(5, 9), sigma=(0.1, 5)),
    np.float32,
    transforms.ToTensor(),
    fixed_image_standardization
])

data_augm_horz_vert = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.RandomHorizontalFlip(p=1),
    transforms.RandomVerticalFlip(p=1),
    np.float32,
    transforms.ToTensor(),
    fixed_image_standardization
])

data_augm_horz_blur = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.RandomHorizontalFlip(p=1),
    transforms.GaussianBlur(kernel_size=(5, 9), sigma=(0.1, 5)),
    np.float32,
    transforms.ToTensor(),
    fixed_image_standardization
])

data_augm_vert_blur = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.RandomVerticalFlip(p=1),
    transforms.GaussianBlur(kernel_size=(5, 9), sigma=(0.1, 5)),
    np.float32,
    transforms.ToTensor(),
    fixed_image_standardization
])

data_augm_horz_vert_blur = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.RandomHorizontalFlip(p=1),
    transforms.RandomVerticalFlip(p=1),
    transforms.GaussianBlur(kernel_size=(5, 9), sigma=(0.1, 5)),
    np.float32,
    transforms.ToTensor(),
    fixed_image_standardization
])

In [4]:
train_dataset = siamese_dataset(data_dir, shuffle_pairs=True)
val_dataset = siamese_dataset(val_dir, shuffle_pairs=False)

In [5]:
train_loader = DataLoader(train_dataset, batch_size=32)
val_loader = DataLoader(val_dataset, batch_size=32)

In [6]:
siamese = SiameseInceptionResnetV1(
    classify=False,
    pretrained='vggface2',
).to(device)

In [40]:
x = next(iter(train_loader))
x

[[tensor([[[[ 0.3867,  0.3867,  0.3867,  ...,  0.3867,  0.3867,  0.3867],
            [ 0.3867,  0.3867,  0.3867,  ...,  0.3867,  0.3867,  0.3867],
            [ 0.4023,  0.4023,  0.4023,  ...,  0.3867,  0.3789,  0.3789],
            ...,
            [ 0.3867,  0.3867,  0.3945,  ...,  0.3867,  0.3867,  0.3867],
            [ 0.3867,  0.3867,  0.3945,  ...,  0.3789,  0.3711,  0.3789],
            [ 0.3945,  0.3945,  0.3945,  ...,  0.3789,  0.3711,  0.3711]],
  
           [[ 0.3945,  0.3945,  0.3945,  ...,  0.4023,  0.4023,  0.4023],
            [ 0.3867,  0.3945,  0.3945,  ...,  0.4023,  0.4023,  0.4023],
            [ 0.3945,  0.3945,  0.3945,  ...,  0.4023,  0.3945,  0.3945],
            ...,
            [ 0.3789,  0.3789,  0.3867,  ...,  0.3867,  0.3867,  0.3867],
            [ 0.3789,  0.3789,  0.3867,  ...,  0.3789,  0.3789,  0.3789],
            [ 0.3867,  0.3867,  0.3867,  ...,  0.3867,  0.3867,  0.3867]],
  
           [[ 0.2539,  0.2539,  0.2539,  ...,  0.3008,  0.3008,  0.300

In [7]:
print(siamese)

SiameseInceptionResnetV1(
  (conv2d_1a): BasicConv2d(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (conv2d_2a): BasicConv2d(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (conv2d_2b): BasicConv2d(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (maxpool_3a): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2d_3b): BasicConv2d(
    (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (conv2

Define optimizer and scheduler

In [9]:
optimizer = optim.Adam(siamese.parameters(), lr=0.001)
scheduler = MultiStepLR(optimizer, [5, 10])

### Define loss and evaluation functions

In [10]:
# loss_fn = torch.nn.CrossEntropyLoss()
loss_fn = torch.nn.BCELoss()
metrics = {
    'fps': training.BatchTimer(),
    'acc': training.accuracy
}

In [12]:
# for i_batch, ((x1, x2), y, (class1, class2)) in enumerate(train_loader):
#     print(y)

In [27]:
x1.shape

torch.Size([32, 3, 256, 256])

In [13]:
y_pred = siamese(x1, x2)

In [14]:
x1.shape

torch.Size([32, 3, 256, 256])

In [25]:
torch.transpose(y, 0, 1)[0][0]

tensor(1.)

In [14]:
y_pred

tensor([[0.0400, 0.0771, 0.0691,  ..., 0.0203, 0.0129, 0.0392],
        [0.0272, 0.0084, 0.0709,  ..., 0.0328, 0.0036, 0.0016],
        [0.0659, 0.0359, 0.0602,  ..., 0.0716, 0.0102, 0.0472],
        ...,
        [0.0224, 0.0054, 0.0391,  ..., 0.1032, 0.0328, 0.0092],
        [0.0085, 0.0873, 0.1174,  ..., 0.0889, 0.0205, 0.0093],
        [0.0918, 0.0710, 0.0176,  ..., 0.0448, 0.0337, 0.0460]],
       grad_fn=<AbsBackward0>)

In [17]:
y[0]

tensor([1.])

In [16]:
loss_fn(y_pred, torch.transpose(y, 0, 1))

ValueError: Expected input batch_size (32) to match target batch_size (1).

### Train model

In [25]:
writer = SummaryWriter()
writer.iteration, writer.interval = 0, 10
best_val = 10000000

print('\n\nInitial')
print('-' * 10)
siamese.eval()
pass_epoch(
    siamese, loss_fn, val_loader,
    batch_metrics=metrics, show_running=True, device=device,
    writer=writer
)

for epoch in range(epochs):
    print('\nEpoch {}/{}'.format(epoch + 1, epochs))
    print('-' * 10)

    # Train
    siamese.train()
    pass_epoch(
        siamese, loss_fn, train_loader, optimizer, scheduler,
        batch_metrics=metrics, show_running=True, device=device,
        writer=writer
    )

    # Validation
    siamese.eval()
    answ = pass_epoch(
        siamese, loss_fn, val_loader,
        batch_metrics=metrics, show_running=True, device=device,
        writer=writer
    )
    loss = answ[0]
    # Save model
    if loss < best_val:
        best_val = loss
        print('Saving best weights')
        torch.save(
            {
                "epoch": epoch + 1,
                "model_state_dict": siamese.state_dict(),
                # "backbone": args.backbone,
                "optimizer_state_dict": optimizer.state_dict()
            },
            os.path.join(model_dir, "best.pth")
        )            

    # Save model based on the frequency defined by "args.save_after"
    if (epoch + 1) % 2 == 0:
        torch.save(
            {
                "epoch": epoch + 1,
                "model_state_dict": siamese.state_dict(),
                # "backbone": args.backbone,
                "optimizer_state_dict": optimizer.state_dict()
            },
            os.path.join(model_dir, "epoch_{}.pth".format(epoch + 1))
        ) 

writer.close()



Initial
----------


ValueError: Expected input batch_size (32) to match target batch_size (1).

In [81]:
torch.save(siamese, 'C:/Users/U339700/Documents/Palas/models/nuevo.pth')